## Training

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('LoanStats.csv')

df = df.loc[:, df.isnull().sum()< 0.9*df.shape[0]]
df = df.dropna(how='all')

df = df.loc[:,df.apply(pd.Series.nunique) != 1]
df = df.drop(['emp_title','title','desc'], axis=1)

#Label-encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sub_grade']=le.fit_transform(df['sub_grade'])
df['addr_state']=le.fit_transform(df['addr_state'])

df['zip_code'] = df['zip_code'].str.replace('xx', '').astype('int')
df['zip_code']=le.fit_transform(df['zip_code'])

#one-hot encoding
df = pd.concat([df, pd.get_dummies(df[['term','grade','home_ownership','verification_status','purpose','debt_settlement_flag']])], axis=1)
df = df.drop(['term','grade','home_ownership','verification_status','purpose','debt_settlement_flag'], axis=1)

# emp-length encoding
look_up = {'1 year':1,'2 years':2,'3 years':3,'4 years':4,'5 years':5,'6 years':6,'7 years':7,'8 years':8,
           '9 years':9,'10 years':10,'10+ years':11,'< 1 year':0}
df['emp_length'] = df['emp_length'].apply(lambda x: look_up[x] if(str(x) != 'nan') else x)

dt_cols = ['issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d']
df['issue_d_month'], df['issue_d_year'] = df['issue_d'].str.split('-', 1).str
df['earliest_cr_line_month'], df['earliest_cr_line_year'] = df['earliest_cr_line'].str.split('-', 1).str
df['last_pymnt_d_month'], df['last_pymnt_d_year'] = df['last_pymnt_d'].str.split('-', 1).str
df['last_credit_pull_d_month'], df['last_credit_pull_d_year'] = df['last_credit_pull_d'].str.split('-', 1).str
df = df.drop(dt_cols, axis=1)
dt_cols_new = ['issue_d_month','issue_d_year','earliest_cr_line_month','earliest_cr_line_year','last_pymnt_d_month',
               'last_pymnt_d_year','last_credit_pull_d_month','last_credit_pull_d_year']
look_up = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
df['issue_d_month'] = df['issue_d_month'].apply(lambda x: look_up[x] if(str(x) != 'nan') else x)
df['earliest_cr_line_month'] = df['earliest_cr_line_month'].apply(lambda x: look_up[x] if(str(x) != 'nan') else x)
df['last_pymnt_d_month'] = df['last_pymnt_d_month'].apply(lambda x: look_up[x] if(str(x) != 'nan') else x)
df['last_credit_pull_d_month'] = df['last_credit_pull_d_month'].apply(lambda x: look_up[x] if(str(x) != 'nan') else x)
df[dt_cols_new] = df[dt_cols_new].astype(float)
df['int_rate'] = df['int_rate'].str.replace('%', '').astype('float')
df['revol_util'] = df['revol_util'].str.replace('%', '').astype('float')
df = df.rename(index=str, columns={'int_rate':'int_rate%', 'revol_util':'revol_util%'})

#impute the missing values
from sklearn.preprocessing import Imputer
imputer = Imputer()
df['tax_liens']=imputer.fit_transform(df[['tax_liens']].values)
df['inq_last_6mths']=imputer.fit_transform(df[['inq_last_6mths']].values)
df['acc_now_delinq']=imputer.fit_transform(df[['acc_now_delinq']].values)
df['pub_rec']=imputer.fit_transform(df[['pub_rec']].values)
df['open_acc']=imputer.fit_transform(df[['open_acc']].values)
df['annual_inc']=imputer.fit_transform(df[['annual_inc']].values)
df['delinq_2yrs']=imputer.fit_transform(df[['delinq_2yrs']].values)
df['total_acc']=imputer.fit_transform(df[['total_acc']].values)
df['delinq_amnt']=imputer.fit_transform(df[['delinq_amnt']].values)
df['mths_since_last_delinq']=imputer.fit_transform(df[['mths_since_last_delinq']].values)
df['pub_rec_bankruptcies']=imputer.fit_transform(df[['pub_rec_bankruptcies']].values)
df['revol_util%']=imputer.fit_transform(df[['revol_util%']].values)
df['emp_length']=imputer.fit_transform(df[['emp_length']].values)

df['last_pymnt_d_year']=imputer.fit_transform(df[['last_pymnt_d_year']].values)
df['last_pymnt_d_month']=imputer.fit_transform(df[['last_pymnt_d_month']].values)
df['earliest_cr_line_year']=imputer.fit_transform(df[['earliest_cr_line_year']].values)
df['earliest_cr_line_month']=imputer.fit_transform(df[['earliest_cr_line_month']].values)
df['last_credit_pull_d_month']=imputer.fit_transform(df[['last_credit_pull_d_month']].values)
df['last_credit_pull_d_year']=imputer.fit_transform(df[['last_credit_pull_d_year']].values)

df.replace({'loan_status': {'Fully Paid': 1, 'Charged Off': 2,
                            'Does not meet the credit policy. Status:Fully Paid':3,
                            'Does not meet the credit policy. Status:Charged Off':4}}, inplace = True)
y = df['loan_status']; X = df.drop('loan_status', axis=1); 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Pickling

In [6]:
from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(decision_tree, 'clf_model.pkl') 

['clf_model.pkl']